In [35]:
import pandas as pd
import json

In [36]:
df = pd.read_csv('../semEval16/check_correctness/graph-train-correctness.csv', encoding='utf-8')
df[df.Target_matching == True]
# df

,sent_id,text,Source,Target,Polar_expression,Polarity,Target_matching
7,1014458:0,"I have eaten at Saul, many times, the food is ...",NaN,the food,"always consistently, outrageously good.",Positive,True
9,1014458:2,The duck confit is always amazing and the foie...,NaN,The duck confit,always amazing,Positive,True
10,1014458:3,The wine list is interesting and has many good...,NaN,The wine list,interesting,Positive,True
16,1016296:3,"Food was okay, nothing great.",NaN,Food,"okay, nothing great.",Positive,True
30,1041457:2,The food is very average...the Thai fusion stu...,NaN,The food,very average...the,Positive,True
...,...,...,...,...,...,...,...
2537,FF#6:5,The hostess was rude and I got a distinct feel...,NaN,The hostess,rude,Negative,True
2544,FF#6:12,Shame on this place for the horrible rude staf...,NaN,staff,rude,Positive,True
2545,FF#10:0,bad staff,NaN,staff,bad,Positive,True
2547,FF#10:2,The food is good.,NaN,The food,good.,Positive,True


In [37]:
duplicateRows = df[df.duplicated()]
duplicateRows

,sent_id,text,Source,Target,Polar_expression,Polarity,Target_matching
494,1349391:1,"sometimes i get bad food and bad service, some...",NaN,"service, sometimes",bad,Positive,True
873,1632445:6,The lobster sandwich is $24 and although it wa...,NaN,it,good,Positive,False
1575,713152:0,"Excellent dumplings served amid clean, chic de...",NaN,dumplings,Excellent,Positive,True
2209,CLF#3:32,I wish I COULD be refunded!,I,NaN,COULD be refunded!,Negative,False


In [38]:
# open training data or test data
with open('../semEval16/graph_parser/predictions/train/opener_en_prediction.json') as f:
    json_data = json.load(f)

json_data

[{'sent_id': '1004293:0',
  'text': 'Judging from previous posts this used to be a good place, but not any longer.',
  'opinions': [{'Source': [[], []],
    'Target': [[], []],
    'Polar_expression': [['not any longer.'], ['62:77']],
    'Polarity': 'Negative',
    'Intensity': 'Standard'}]},
 {'sent_id': '1004293:1',
  'text': 'We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.',
  'opinions': [{'Source': [[], []],
    'Target': [[], []],
    'Polar_expression': [['-'], ['65:66']],
    'Polarity': 'Positive',
    'Intensity': 'Standard'},
   {'Source': [[], []],
    'Target': [[], []],
    'Polar_expression': [['there were'], ['4:14']],
    'Polarity': 'Positive',
    'Intensity': 'Standard'}]},
 {'sent_id': '1004293:2',
  'text': 'They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.',
  'opinions': []},
 {'sent_id': '1004293:3',
 

In [39]:
# assign 'Matching' to json based on correctness(df)
count = 0

for i in json_data:
    sid = i['sent_id']

    for j in i['opinions']:
        j['Matching'] = False

        if len(j['Target'][0]) > 0:
            tar = j['Target'][0][0]
            exp = j['Polar_expression'][0][0]
            df2 = df[df.sent_id == sid]
            # print(df2)

            for k in range(len(df2)):
                # match pairs of json and csv
                if (pd.isnull(df2.iloc[k]['Target']) == False) and (df2.iloc[k]['Target'] == tar) and (df2.iloc[k]['Polar_expression'] == exp):
                    if (df2.iloc[k]['Target_matching'] == True):
                        j['Matching'] = True
                        count += 1
                        break
                        # print(sid, df2.iloc[k]['Target'], df2.iloc[k]['Polar_expression'])       
        

print(f"number of text: {len(json_data)}")
print(f'number of true targets: {count}')

number of text: 2000
number of true targets: 924


In [40]:
# deep copy original json file
import copy
json_list = copy.deepcopy(json_data)

In [41]:
# check true pairs in json 
true_df = df[df.Target_matching == True]

wrong = 0
sid_true = []
for s in range(len(true_df)):
    flag = False

    for t in range(len(json_list)):
        for r in json_list[t]['opinions']:
            if (r['Matching']) and (true_df.iloc[s]['sent_id'] == json_list[t]['sent_id']):
                if (true_df.iloc[s]['Target'] == r['Target'][0][0]) and (true_df.iloc[s]['Polar_expression'] == r['Polar_expression'][0][0]):
                    flag = True
                    break
        if flag == True:
            break


    if flag == False:
        wrong += 1
        sid_true.append(true_df.iloc[s]['sent_id'])
        print(true_df.iloc[s]['sent_id'] + ', ' + true_df.iloc[s]['Target'] + ', ' + true_df.iloc[s]['Polar_expression'])

print(f'number of missing pairs: {wrong}')

1632445:6, The lobster sandwich, $24 
Z#9:0, sweet  seating, short
Z#9:0, sweet  seating, great:it's romantic,cozy
Z#9:0, sweet  seating, private.
number of missing pairs: 4


In [42]:
# correct missing true pairs manually

for x in range(len(json_list)):
    if json_list[x]['sent_id'] in sid_true:

        for y in range(len(json_list[x]['opinions'])):
            print(x, y, json_list[x]['opinions'][y])
            print('--------------')

# edit 'Matching' manually
json_list[662]['opinions'][3]['Matching'] = True
json_list[1517]['opinions'][0]['Matching'] = True
json_list[1517]['opinions'][1]['Matching'] = True
json_list[1517]['opinions'][2]['Matching'] = True

662 0 {'Source': [[], []], 'Target': [['it'], ['53:55']], 'Polar_expression': [['not nearly enough'], ['60:77']], 'Polarity': 'Negative', 'Intensity': 'Standard', 'Matching': False}
--------------
662 1 {'Source': [[], []], 'Target': [['it'], ['41:43']], 'Polar_expression': [['good'], ['48:52']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': False}
--------------
662 2 {'Source': [[], []], 'Target': [['it'], ['53:55']], 'Polar_expression': [['good'], ['48:52']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': False}
--------------
662 3 {'Source': [[], []], 'Target': [['The lobster sandwich'], ['0:20']], 'Polar_expression': [['$24'], ['24:27']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': False}
--------------
1517 0 {'Source': [[], []], 'Target': [['sweet – seating'], ['10:25']], 'Polar_expression': [['short'], ['0:5']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': False}
--------------
1517 1 {'Source': [[], []], 'Target': 

In [43]:
# check numbe of true pairs

count = 0
for i in range(len(json_list)):
        for j in json_list[i]['opinions']:
            if j['Matching']:
                count += 1


print(f"number of text: {len(json_list)}")
print(f'number of true targets: {count}')

number of text: 2000
number of true targets: 928


In [44]:
# remove all false pairs

for i in range(len(json_list)):
    new_list = [j for j in json_list[i]['opinions'] if j['Matching']]
    json_list[i]['opinions'] = new_list
        
json_list

[{'sent_id': '1004293:0',
  'text': 'Judging from previous posts this used to be a good place, but not any longer.',
  'opinions': []},
 {'sent_id': '1004293:1',
  'text': 'We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.',
  'opinions': []},
 {'sent_id': '1004293:2',
  'text': 'They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.',
  'opinions': []},
 {'sent_id': '1004293:3',
  'text': 'The food was lousy - too sweet or too salty and the portions tiny.',
  'opinions': []},
 {'sent_id': '1004293:4',
  'text': 'After all that, they complained to me about the small tip.',
  'opinions': []},
 {'sent_id': '1004293:5', 'text': 'Avoid this place!', 'opinions': []},
 {'sent_id': '1014458:0',
  'text': 'I have eaten at Saul, many times, the food is always consistently, outrageously good.',
  'opinions': [{'Source': [[], []],
    'Target': 

In [45]:
# remove all empty opinions
json_new = []

for i in range(len(json_list)):
    if len(json_list[i]['opinions']) > 0:
        json_new.append(json_list[i])

print(f'nember of text: {len(json_new)}')


nember of text: 623


In [46]:
# add key of dataset

for i in range(len(json_new)):
    json_new[i]['dataset'] = 's16+_train'

json_new

[{'sent_id': '1014458:0',
  'text': 'I have eaten at Saul, many times, the food is always consistently, outrageously good.',
  'opinions': [{'Source': [[], []],
    'Target': [['the food'], ['34:42']],
    'Polar_expression': [['always consistently, outrageously good.'],
     ['46:85']],
    'Polarity': 'Positive',
    'Intensity': 'Standard',
    'Matching': True}],
  'dataset': 's16+_train'},
 {'sent_id': '1014458:2',
  'text': 'The duck confit is always amazing and the foie gras terrine with figs was out of this world.',
  'opinions': [{'Source': [[], []],
    'Target': [['The duck confit'], ['0:15']],
    'Polar_expression': [['always amazing'], ['19:33']],
    'Polarity': 'Positive',
    'Intensity': 'Standard',
    'Matching': True}],
  'dataset': 's16+_train'},
 {'sent_id': '1014458:3',
  'text': 'The wine list is interesting and has many good values.',
  'opinions': [{'Source': [[], []],
    'Target': [['The wine list'], ['0:13']],
    'Polar_expression': [['interesting'], ['17

In [52]:
# convert to json file

with open('../semEval16_plus/s16+_train.json', 'w') as outfile:
    json.dump(json_new, outfile)